In [261]:
import glob
import cv2
import numpy as np


In [308]:
# Define the size of the calibration pattern (chessboard)
chessboard_size = (7, 5)  

# Define the actual dimensions of the square in millimeters
square_size_mm = 27  


In [309]:
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size_mm


In [310]:
# Arrays to store object points and image points from all the images
obj_points = []  # 3D points in real world space
img_points = []  # 2D points in image plane


In [311]:
# Find images in the calibration image directory
calibration_images = glob.glob('captured_images/*.jpg')


In [312]:
# Loop through each calibration image
for image_path in calibration_images:
    # Load the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        obj_points.append(objp)
        corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        img_points.append(corners_refined)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, chessboard_size, corners_refined, ret)
        cv2.imshow('Chessboard Corners', img)
        cv2.waitKey(500)  # Show each image for 0.5 seconds
        cv2.destroyAllWindows()


In [313]:
# Perform camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)




In [320]:
for fname in calibration_images:
        img = cv2.imread(fname)
        h,  w = img.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

        # undistort
        dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

        # crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]
        
        # save new image into file
        #cv2.imwrite(f"{fname.split('.')[0]}_result.png", dst)

In [322]:
roi

(6, 23, 1872, 1015)

In [323]:
# Print calibration matrix
print("Calibration Matrix (Intrinsic Parameters):\n", mtx)

Calibration Matrix (Intrinsic Parameters):
 [[2.02100117e+03 0.00000000e+00 9.28608280e+02]
 [0.00000000e+00 2.04959679e+03 4.87634015e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [269]:
# Select one image for calibration
img_path = 'captured_images/image_2.jpg'
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [270]:
# Define the chessboard size
chessboard_size = (7, 5)  # Change according to your chessboard

In [271]:
# Find chessboard corners
ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

In [272]:
if ret:
    # Draw and display the corners
    cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
    cv2.imshow('Corners', img)
    cv2.waitKey(500)
    cv2.destroyAllWindows()
    
    print(" chalo ye to hua ")

    # Define 3D world points assuming a known square size
    square_size = 27.0  # Change according to your chessboard
    obj_points = np.zeros((np.prod(chessboard_size), 3), dtype=np.float32)
    obj_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size
    print("  ye to hona hi tha ")
    # Convert corners to float32
    corners = corners.astype(np.float32)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([obj_points], [corners], gray.shape[::-1], None, None)

    # Compute rotation matrix and angles
    print(" bhot kuch hogya ")
    rmat, _ = cv2.Rodrigues(rvecs[0])
    print(" bss hone hi wala hai")
    _, _, _, _, _, angles, _ = cv2.decomposeProjectionMatrix(np.hstack((rmat, tvecs[0])))

    print("Intrinsic Matrix:")
    print(mtx)
    print("---------------------------------------------------------------------")
    print("Extrinsic Rotation Matrix:")
    print(rmat)
    print("---------------------------------------------------------------------")
    print("Extrinsic Translation Vector:")
    print(tvecs[0])
    print("---------------------------------------------------------------------")
    print("Rotation angles (degrees) around x, y, z axes:")
    print(np.degrees(angles))

else:
    print("Chessboard corners not found.")


 chalo ye to hua 
  ye to hona hi tha 
 bhot kuch hogya 
 bss hone hi wala hai
Intrinsic Matrix:
[[2.44001303e+03 0.00000000e+00 8.65801079e+02]
 [0.00000000e+00 2.60315904e+03 6.79341433e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
---------------------------------------------------------------------
Extrinsic Rotation Matrix:
[[ 0.99764599  0.05910176  0.03477718]
 [-0.06819471  0.90837452  0.41255934]
 [-0.00720772 -0.41395979  0.91026663]]
---------------------------------------------------------------------
Extrinsic Translation Vector:
[[ -44.93384735]
 [-113.31685744]
 [ 525.20444887]]
---------------------------------------------------------------------
Rotation angles (degrees) around x, y, z axes:
[[57.16238976 -3.9073708   0.        ]
 [ 3.9073708  57.16238976  0.        ]
 [ 0.          0.         57.29577951]]


In [273]:
fx = mtx[0, 0]  # Focal length along the x-axis
fy = mtx[1, 1]  # Focal length along the y-axis
cx = mtx[0, 2]
cy = mtx[1, 2]


In [274]:
(fx+fy) / 2

2521.586031352658

In [275]:
cx

865.8010789965606

In [276]:
cy

679.341433383635

In [277]:
cv2.destroyAllWindows()

In [278]:
import numpy as np

def rotation_matrix_to_euler_angles(rotation_matrix):
    # Extract rotation angles from rotation matrix
    # Note: Assumes rotation_matrix is a valid 3x3 rotation matrix
    
    # Calculate theta_x (rotation angle around X-axis)
    theta_x = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
    
    # Calculate theta_y (rotation angle around Y-axis)
    theta_y = np.arctan2(-rotation_matrix[2, 0], np.sqrt(rotation_matrix[2, 1]**2 + rotation_matrix[2, 2]**2))
    
    # Calculate theta_z (rotation angle around Z-axis)
    theta_z = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    
    # Convert angles from radians to degrees
    theta_x_deg = np.degrees(theta_x)
    theta_y_deg = np.degrees(theta_y)
    theta_z_deg = np.degrees(theta_z)
    
    return theta_x_deg, theta_y_deg, theta_z_deg

# Example rotation matrix
rotation_matrix = rmat

# Calculate rotation angles
theta_x_deg, theta_y_deg, theta_z_deg = rotation_matrix_to_euler_angles(rotation_matrix)

# Print the rotation angles
print("Rotation Angles (degrees) around X, Y, Z axes:", theta_x_deg, theta_y_deg, theta_z_deg)


Rotation Angles (degrees) around X, Y, Z axes: -24.454503696489343 0.4129754591803978 -3.910405866182276


In [279]:
import cv2
import numpy as np


In [280]:
import cv2
import numpy as np

# Function to compute object dimensions using perspective projection equations
def compute_object_dimensions(image_width, image_height, object_width_in_pixels, object_height_in_pixels, distance_to_object):
    # Define the focal length (in pixels) - approximate value
    focal_length = (image_width + image_height) / 2
    
    # Compute object dimensions in real-world units (e.g., meters)
    object_width = (object_width_in_pixels * distance_to_object) / focal_length
    object_height = (object_height_in_pixels * distance_to_object) / focal_length
    
    return object_width, object_height




In [281]:
def capture_image(camera_index=0):
    cap = cv2.VideoCapture(camera_index)
    ret, frame = cap.read()
    cap.release()
    return frame

# Example usage


In [282]:
import cv2
import numpy as np

def get_object_dimensions_from_image(image, threshold_method=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, block_size=11, c=2):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding or other preprocessing as needed
    thresh = cv2.adaptiveThreshold(gray, 255, threshold_method, cv2.THRESH_BINARY_INV, block_size, c)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Select the largest contour (assuming it corresponds to the object)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Get the bounding rectangle of the contour
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Draw bounding box
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Compute object dimensions in pixels
    object_width_in_pixels = w
    object_height_in_pixels = h
    
    return object_width_in_pixels, object_height_in_pixels, image


In [284]:
def main():
    # Known parameters
    distance_to_object = 280  # Distance from the camera to the object in millimeters
    
    # Capture an image
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    
    # Get image dimensions
    image_height, image_width, _ = frame.shape
    print("Image dimensions (width x height) in pixels:", image_height, "x", image_width)

    
    object_width_in_pixels, object_height_in_pixels, image_with_box = get_object_dimensions_from_image(frame)
    print("Object dimensions (width x height) in pixels:", object_width_in_pixels, "x", object_height_in_pixels)


    # Compute object dimensions
    object_width, object_height = compute_object_dimensions(image_width, image_height, object_width_in_pixels, object_height_in_pixels, distance_to_object)
    
    
    # Display the image with bounding box
    cv2.imshow('Object with Bounding Box', image_with_box)
    cv2.waitKey(50)
    cv2.destroyAllWindows()
    print("Object dimensions (width x height):", object_width, "mm x", object_height, "mm")

if __name__ == "__main__":
    main()


Image dimensions (width x height) in pixels: 1080 x 1920
Object dimensions (width x height) in pixels: 420 x 251
Object dimensions (width x height): 78.4 mm x 46.85333333333333 mm


In [301]:
import cv2
import numpy as np

def calibrate_camera(image_paths, pattern_size):
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    #objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    #objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
    objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size_mm

    # Arrays to store object points and image points from all the images.
    object_points = []  # 3D points in real world space
    image_points = []   # 2D points in image plane.

    # Step through the list and search for chessboard corners
    for path in image_paths:
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

        # If found, add object points, image points
        if ret:
            object_points.append(objp)
            image_points.append(corners)

    # Calibrate the camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, gray.shape[::-1], None, None)

    return ret, mtx, dist, rvecs, tvecs



In [316]:

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

    # Get image size
img_shape = image.shape[::-1]

    # Compute the camera matrix and distortion coefficients
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, img_shape, 1)

    # Undistort the image
undistorted_img = cv2.undistort(image, mtx, dist, None, new_camera_mtx)

    # Convert image to grayscale
gray = cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
ret, corners = cv2.findChessboardCorners(gray, img_points, None)

    # If found, refine the corners
if ret:
    corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

        # Compute the extrinsic matrix
    _, rvecs, tvecs = cv2.solvePnP(obj_size, corners_refined, mtx, dist)
    r_mat, _ = cv2.Rodrigues(rvecs)
        
     # Calculate the object dimensions
    _, _, object_dims, _ = cv2.decomposeProjectionMatrix(np.hstack((r_mat, tvecs)))
    object_width = object_dims[0]
    object_height = object_dims[1]




error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'getOptimalNewCameraMatrix'
> Overload resolution failed:
>  - Can't parse 'imageSize'. Expected sequence length 2, got 3
>  - Can't parse 'imageSize'. Expected sequence length 2, got 3


In [305]:
# Example usage
if __name__ == "__main__":
    # Example object dimensions (in meters)

    # Example image paths and chessboard pattern size
    image_paths = calibration_images  # List of image paths
    pattern_size = (7, 5)  # Chessboard pattern size

    # Load the images and calculate object dimensions
    for path in image_paths:
        img = cv2.imread(path)
        width, height = calculate_dimensions(img, obj_size, pattern_size,0.027)
        print(f"Dimensions of the object in {path}: Width = {width} meters, Height = {height} meters")


UnboundLocalError: cannot access local variable 'gray' where it is not associated with a value

In [99]:
print(type(boardSize))

<class 'numpy.ndarray'>


In [324]:
import numpy as np
import cv2
import glob

# Termination criteria for the corner sub-pixel refinement process
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0), ..., (6,5,0)
# Adjust the following parameters according to your chessboard
checkerboard_size = (7, 5)
objp = np.zeros((checkerboard_size[0]*checkerboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images
objpoints = []  # 3d points in real-world space
imgpoints = []  # 2d points in image plane

# List of calibration images
images = glob.glob('captured_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, checkerboard_size, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [342]:
import cv2
import numpy as np

# Load the chessboard image
img = cv2.imread('captured_images/image_2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Define the chessboard size
chessboard_size = (7, 7)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

# If found, refine the corner positions
if ret:
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    
    # Draw and display the corners
    cv2.drawChessboardCorners(img, chessboard_size, corners2, ret)
    cv2.imshow('Corners', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [343]:
# Assuming corners2 is sorted row-wise, we select the four outer corners
p1 = corners2[0].ravel()  # Top-left corner in the image
p2 = corners2[6].ravel()  # Top-right corner in the image
p3 = corners2[-1].ravel()  # Bottom-right corner in the image
p4 = corners2[-7].ravel()  # Bottom-left corner in the image

# 2D image points
img_points = np.array([p1, p2, p3, p4])

# 3D real-world points
obj_points = np.array([[0, 0, 0], [120, 0, 0], [120, 120, 0], [0, 120, 0]], dtype='float32')

In [344]:
ret, rvecs, tvecs = cv2.solvePnP(obj_points, img_points, mtx, dist)

# Convert rotation vectors to rotation matrix
R_mtx, jac = cv2.Rodrigues(rvecs)

# Now, R_mtx is the rotation matrix and tvecs is the translation vector

In [347]:
R_mtx

array([[ 0.99900082, -0.04141265,  0.01680367],
       [ 0.01733315,  0.70557862,  0.70841963],
       [-0.04119384, -0.70742053,  0.70559143]])

In [345]:
import math

def rotation_matrix_to_euler_angles(R):
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])

# Assuming R_mtx is the rotation matrix obtained earlier
euler_angles = rotation_matrix_to_euler_angles(R_mtx)

In [346]:
euler_angles

array([-0.78669263,  0.0412055 ,  0.01734875])

In [325]:
mtx

array([[2.02100114e+03, 0.00000000e+00, 9.28608282e+02],
       [0.00000000e+00, 2.04959675e+03, 4.87634022e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [326]:
def measure_object_dimension(frame, mtx, dist):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Blur the image to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection
    edged = cv2.Canny(blur, 35, 125)
    
    # Find contours
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours based on their area, keeping only the largest
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
    
    for c in contours:
        # Approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        # If the approximated contour has four points, assume it's a rectangle (or similar)
        if len(approx) == 4:
            # Draw the contour
            cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)
            
            # Compute the bounding box of the contour and use it to compute the object dimensions
            (x, y, w, h) = cv2.boundingRect(approx)
            width = w  # Placeholder for actual dimension calculation
            height = h  # Placeholder for actual dimension calculation
            
            # Display the dimensions on the image
            cv2.putText(frame, f"Width: {width} pixels", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(frame, f"Height: {height} pixels", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

In [337]:
def measure_object_dimension(frame, mtx, dist, distance_to_object):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Blur the image to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection
    edged = cv2.Canny(blur, 35, 125)
    
    # Find contours
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours based on their area, keeping only the largest
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
    

    for c in contours:
        # Approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        
        # If the approximated contour has four points, assume it's a rectangle (or similar)
        if len(approx) == 4:
            # Draw the contour
            cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)
            
            # Compute the bounding box of the contour and use it to compute the object dimensions
            (x, y, w, h) = cv2.boundingRect(approx)
            width_pixels = w  # Placeholder for actual dimension calculation
            height_pixels = h  # Placeholder for actual dimension calculation

        # Calculate real-world dimensions
            width_mm = (width_pixels * distance_to_object) / mtx[0, 0]  # Using f_x from the camera matrix
            height_mm = (height_pixels * distance_to_object) / mtx[1, 1]  # Using f_y from the camera matrix

            # Display the dimensions on the image
            cv2.putText(frame, f"Width: {width_mm:.2f} mm", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(frame, f"Height: {height_mm:.2f} mm", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

In [341]:
import cv2


    

    # Start video capture
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    
        # Capture frame-by-frame
    ret, frame = cap.read()

        # If frame is read correctly, ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

        # Call the object measurement function on the current frame
    measure_object_dimension(frame, mtx, dist,350)

        # Display the resulting frame
    cv2.imshow('frame', frame)

        # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

    # When everything done, release the capture
cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 